In [7]:
! gdown https://drive.google.com/uc?id=1pGDcwitiFXy-CY9vO3wH3XprwjWgsQ8R

Downloading...
From (original): https://drive.google.com/uc?id=1pGDcwitiFXy-CY9vO3wH3XprwjWgsQ8R
From (redirected): https://drive.google.com/uc?id=1pGDcwitiFXy-CY9vO3wH3XprwjWgsQ8R&confirm=t&uuid=358c192f-2ba0-44e9-81d7-bd3d9a054de6
To: F:\ML_notebooks\z.Gen_vs_Contrastive\Contrastive\semi_super_augPipe.py

  0%|          | 0.00/12.2k [00:00<?, ?B/s]
100%|##########| 12.2k/12.2k [00:00<?, ?B/s]


In [10]:
cifar10 = tf.keras.datasets.cifar10
 
# Distribute it to train and test set
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

(50000, 32, 32, 3) (50000, 1) (10000, 32, 32, 3) (10000, 1)


In [8]:
IMG_H, IMG_W = (32,32)
input_shape = (IMG_H,IMG_W,3)
BATCH_SIZE = 128


# The reason to reduce the aggresive nature of random crop is to get the full picture idea & not the background
min_obj_cov_value = 0.7
color_jitter_value = 0.1

# Model Name for reference
modelNameStr = "MobileNetV1_L2_10_6_warmUp_BYOL_batchNorm_colorJ_" + str(color_jitter_value) + "_op_1024_random_max_pool_minObjCov_"+str(min_obj_cov_value)
print("Model Name : ",modelNameStr)

# Basic Imports
import numpy as np 
import pandas as pd 
import tensorflow as tf

import gc
import time
import pickle
import math
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import Callback, LearningRateScheduler
import matplotlib.pyplot as plt
from imutils import paths
from tqdm import tqdm
import tensorflow as tf
import numpy as np
import cv2
import functools
from semi_super_augPipe import preprocess_image
from typing import Callable
import shutil

# Loss plot
from scipy.ndimage.filters import gaussian_filter1d
import matplotlib.pyplot as plt
import seaborn as sns 
import numpy as np
import random 
plt.style.use('seaborn')

import datetime

# Random seed fixation for experiment result repitition
tf.random.set_seed(10)
np.random.seed(10)

# Directory Setup
import os
import tempfile
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
print("Tensorflow version : ",tf.__version__)

# In-order run function decorators in tf2.0
tf.config.run_functions_eagerly(False)
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)


Model Name :  MobileNetV1_L2_10_6_warmUp_BYOL_batchNorm_colorJ_0.1_op_1024_random_max_pool_minObjCov_0.7


C:\Users\shaif\anaconda3\envs\tens\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Tensorflow version :  2.8.0


In [11]:
# Image preprocessing utils
@tf.function
def train_parse_images(image):

    image = tf.image.convert_image_dtype(image, tf.float32)
    # IMG_H & IMG_W are constants that will be saved in the tf.graph during the tracing step (i.e. the first call)
    image = tf.image.resize(image, size=[IMG_H, IMG_W])

    aug_image_1 = preprocess_image(image = image, 
                                   height = IMG_H, 
                                   width  = IMG_W, 
                                   cjs = color_jitter_value,
                                   m_obj_cov = min_obj_cov_value,
                                   a_range = (min_obj_cov_value, 1.0))
  
    
    aug_image_2 = preprocess_image(image = image, 
                                   height = IMG_H, 
                                   width  = IMG_W, 
                                   cjs = color_jitter_value,
                                   m_obj_cov = min_obj_cov_value,
                                   a_range = (min_obj_cov_value, 1.0))
    
    return aug_image_1, aug_image_2
train_tensor = tf.data.Dataset.from_tensor_slices(x_train)
train_ds_shuffled = train_tensor.shuffle(len(train_tensor))

final_train_ds = (train_ds_shuffled
                .map(train_parse_images, num_parallel_calls=tf.data.experimental.AUTOTUNE)
                .batch(BATCH_SIZE, drop_remainder=True)
                .prefetch(tf.data.experimental.AUTOTUNE))

In [12]:
# adding l2 reg
def add_regularization(model, regularizer = tf.keras.regularizers.l2(0.0001)):
    """
    Function to add l2 regularisation penalty to each layer of a built in keras model.
    Args:
        model (tf.keras.models) : Input model whose layer's you want to regularize
        regularizer ( tf.keras.regularizers.l2) : object indicating l2 penalty to be applied.
    
    Returns:
        l2 regularised keras model
    """

    # Regularizer check 
    if not isinstance(regularizer, tf.keras.regularizers.Regularizer):
        print("Regularizer must be a subclass of tf.keras.regularizers.Regularizer")
        return model
    
    # Apply l2 reg in each layer in which we can apply the penalty
    for layer in model.layers:
        for attr in ['kernel_regularizer']:
            if hasattr(layer, attr):
                setattr(layer, attr, regularizer)

    # When we change the layers attributes, the change only happens in the model config file
    model_json = model.to_json()
    
    # Save the weights before reloading the model.
    # Stragtegy helps us to save pretrained weights if the passed input model has any
    tmp_weights_path = os.path.join(tempfile.gettempdir(), 'tmp_weights.h5')
    model.save_weights(tmp_weights_path)

    # load the model from the config
    model = tf.keras.models.model_from_json(model_json)
    
    # Reload the model weights
    model.load_weights(tmp_weights_path, by_name=True)
    return model
def predictor(input_dim, 
              l2_reg_penalty = 1.5*10e-6, 
              hid_1 = 512, 
              hid_2 = 256):
    """
    Function to build the predictor newtork on top of one the arms of the BYOL framework
    Args:
        input_dim (int) : Input vector dimensionality
        l2_reg_penalty (float) : L2 penalty to be applied.
        hid_1, hid_2 (int) : Hidden layer dimensionality of the predictor n/w
    Returns:
        The keras model of the network
    """
  
    regularizer = tf.keras.regularizers.l2(l2_reg_penalty)
    
    inputs = Input(input_dim)
  
    # Non linear prediction layer for the task to increasing the mutual information between 2 views
    prediction_1 = Dense(hid_1, kernel_regularizer = regularizer)(inputs)
    prediction_1 = tf.keras.layers.BatchNormalization()(prediction_1)
    prediction_1 = Activation("relu")(prediction_1)
    prediction_2 = Dense(hid_2, kernel_regularizer = regularizer)(prediction_1)

    return Model(inputs,prediction_2)

def encoder_projector(l2_reg_penalty = 1.5*10e-6, 
                      input_shape = (224,224,3), 
                      hid_1 = 512, 
                      hid_2 = 256):
    """
    Function to build the symmetric part of (online & offline n/w) the BYOL f/w
    Args:
        l2_reg_penalty (float) : L2 penalty to be applied.
        input_shape (tuple) : input image shape of the model.
        hid_1, hid_2 (int) : Hidden layer dimensionality of the projector n/w only
    """

    # None itself is random initalisation that ensures on each 
    # call the model gets alloted different random weights.
    # Both the online & target branch need to alloted different random weights.
    base_model = tf.keras.applications.MobileNet(include_top = False, 
                                                weights = None, 
                                                input_shape = input_shape)
  
    regularizer = tf.keras.regularizers.l2(l2_reg_penalty)

    # If we use imagenet weights on a pretrained model 
    # we have to use this helper to add l2 penalty in each layer
    reg_base_model = add_regularization(base_model, regularizer)
    reg_base_model.trainable = True

    inputs = Input(input_shape)
    h = reg_base_model(inputs, training=True)
    embed_vector = GlobalMaxPooling2D()(h)

    # since architecturally projector & predictor are the same
    projector_model = predictor(embed_vector.shape[-1], hid_1 = hid_1, hid_2 = hid_2)

    op = projector_model(embed_vector)
    enc_prec = Model(inputs, op)
    return enc_prec

In [13]:
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
train_log_dir = 'byol_logs/gradient_tape/' + current_time + '/train'
test_log_dir = 'byol_logs/gradient_tape/' + current_time + '/test'
train_summary_writer = tf.summary.create_file_writer(train_log_dir)
test_summary_writer = tf.summary.create_file_writer(test_log_dir)

def byol_loss(p, z):
    p = tf.math.l2_normalize(p, axis=1)  # (2*bs, 128)
    z = tf.math.l2_normalize(z, axis=1)  # (2*bs, 128)

    similarities = tf.reduce_sum(tf.multiply(p, z), axis=1)
    return 2 - 2 * tf.reduce_mean(similarities)

@tf.function
def train_step(x1, 
               x2,
               online_enc_proj,
               target_enc_proj,
               online_predictor,
               optimizer,
               loss_func):
        
    #Forward pass
    z_target_1 = target_enc_proj(x1, training=True)
    z_target_2 = target_enc_proj(x2, training=True)

    with tf.GradientTape(persistent=True) as tape:
        z_online_1 = online_enc_proj(x1, training=True)
        p_online_1 = online_predictor(z_online_1, training = True)

        z_online_2 = online_enc_proj(x2, training=True)
        p_online_2 = online_predictor(z_online_2, training = True)

        # Trying to make the loss symmetric in nature
        p_online = tf.concat([p_online_1, p_online_2], axis=0)
        z_target = tf.concat([z_target_2, z_target_1], axis=0)
        loss = loss_func(p_online, z_target)

    # Backward pass (update online networks)
    # encoder + projector update
    grads_1 = tape.gradient(loss, online_enc_proj.trainable_variables)
    optimizer.apply_gradients(zip(grads_1, online_enc_proj.trainable_variables))
 
    # predictor update
    grads_2 = tape.gradient(loss, online_predictor.trainable_variables)
    optimizer.apply_gradients(zip(grads_2, online_predictor.trainable_variables))
    del tape

    return loss, grads_1

def train_byol(online_enc_proj,
               online_predictor,
               target_enc_proj, 
               train_dataset, 
               optimizer, 
               criterion,
               lr_epoch,
               beta_base = 0.996,
               epochs=100,
               num_train_samples_viz = 5,
               num_test_samples_viz = 2):
    
    #################################################
    # Setting up epoch steps & time taken  
    t_start = time.time()
    epoch_wise_loss = []
    print("Number of steps : ",len(train_dataset))
    

    for epoch in range(0, epochs):
        
        cnt = 0
        
        # Reset loss collection each step
        step_wise_loss = []
        num_train_steps = len(train_dataset) 
        
        # Tensor board visualisations 
        random_batches_train = random.sample(range(len(train_dataset)),num_train_samples_viz)
        cnt = 0
        random_collection_train_sample_1 = []
        random_collection_train_sample_2 = []
        gradArray = None
        loss = None 

        #################################################
        # Train STEP
        for step, image_batch in enumerate(tqdm(train_dataset)):
            a = image_batch[0]
            b = image_batch[1]

            loss, gradArray = train_step(a, 
                                         b, 
                                         online_enc_proj, 
                                         target_enc_proj, 
                                         online_predictor, 
                                         optimizer, 
                                         criterion)
            step_wise_loss.append(loss)

            # slow update of the target n/w
            f_target_weights = target_enc_proj.get_weights()
            f_online_weights = online_enc_proj.get_weights()

            # Beta weight uodates ....
            beta_final =  1 - (1 - beta_base)*((np.cos((np.pi*(step+1))/(num_train_steps)) + 1)/2)
            for i in range(len(f_online_weights)):
                f_target_weights[i] = beta_final * f_target_weights[i] + (1 - beta_final) * f_online_weights[i]
            
            # Setting updates weights .....
            target_enc_proj.set_weights(f_target_weights)


            # Logging images for tensorboard:
            if cnt in random_batches_train:
                random_collection_train_sample_1.append(image_batch[0][0])
                random_collection_train_sample_2.append(image_batch[1][0])
            cnt+=1
                
        # Average loss throughout the whole process
        if not len(epoch_wise_loss):
            epoch_wise_loss.append(np.mean(step_wise_loss))
        else:
            # Adding the mean of previous ones
            mean_value = (np.sum(step_wise_loss) + epoch_wise_loss[-1]*(epoch)*num_train_steps)/((epoch+1)*num_train_steps)
            epoch_wise_loss.append(mean_value)
        
      
        #################################################
        # Lr tracking
        lr_epoch.append(optimizer._decayed_lr(tf.float32).numpy())
        
        #################################################
        # Console progress
        print("\n epoch: {} | train loss: {:.8f} | lr : {} | {:.4f} mins"
              .format(epoch + 1,
                      epoch_wise_loss[-1],
                      optimizer._decayed_lr(tf.float32).numpy(), 
                      (time.time()-t_start)/60.0))
        
        #################################################
        # Model Save, you can add an heuristic to save if you want
        if not os.path.isdir(modelNameStr):
            os.mkdir(modelNameStr)
                
        online_enc_proj.save("./" + modelNameStr + "/online_encoder_projector" + ".h5")
        online_predictor.save("./" + modelNameStr + "/online_predictor" +".h5")
        target_enc_proj.save("./" + modelNameStr + "/target_encoder_projector" +".h5")
        
        #################################################
        # Tensorboard logging
        tf.summary.experimental.set_step(epoch)

        with train_summary_writer.as_default():
            tf.summary.scalar('loss', epoch_wise_loss[-1], step = epoch)
            tf.summary.scalar('learningRate', lr_epoch[-1], step = epoch)
            tf.summary.image('View 1', random_collection_train_sample_1, step = epoch)
            tf.summary.image('View 2', random_collection_train_sample_2, step = epoch)
          
            # global variable defined later
            # Observe the gradients distributions ..
            for name in layer_names:
                tf.summary.histogram(name + "_gradients", gradArray[layer_to_index[name]])
                
            # Observe model weights 
            for layer in online_enc_proj.layers:
                for tl in layer.trainable_weights:
                    if tl.name in layer_names:
                        tf.summary.histogram(tl.name+"_weights", tl.numpy())
        gc.collect()

    return epoch_wise_loss

In [15]:
class WarmUp(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self,
               initial_learning_rate: float,
               decay_schedule_fn: Callable,
               warmup_steps: int,
               power: float = 1.0,
               name: str = None,):

        super().__init__()
        self.initial_learning_rate = initial_learning_rate
        self.warmup_steps = warmup_steps
        self.power = power
        self.decay_schedule_fn = decay_schedule_fn
        self.name = name

    def __call__(self, step):
        with tf.name_scope(self.name or "WarmUp") as name:
            # Implements polynomial warmup. i.e., if global_step < warmup_steps, the
            # learning rate will be `global_step/num_warmup_steps * init_lr`.
            global_step_float = tf.cast(step, tf.float32)
            warmup_steps_float = tf.cast(self.warmup_steps, tf.float32)
            warmup_percent_done = global_step_float / warmup_steps_float
            warmup_learning_rate = self.initial_learning_rate * tf.math.pow(warmup_percent_done, self.power)
            return tf.cond(
                global_step_float < warmup_steps_float,
                lambda: warmup_learning_rate,
                lambda: self.decay_schedule_fn(step - self.warmup_steps),
                name=name,
            )

    def get_config(self):
        return {
                "initial_learning_rate": self.initial_learning_rate,
                "decay_schedule_fn": self.decay_schedule_fn,
                "warmup_steps": self.warmup_steps,
                "power": self.power,
                "name": self.name,
                }
    
criterion = byol_loss
decay_steps = (len(final_train_ds))*1000
warmup_steps = (len(final_train_ds))*10
print(modelNameStr)
saving_model_weights = ""
l2_reg_penalty = 1.5*10e-6

# parameters for the hidden layer of projector & predictior (scaled down for fast execution)
hidden_layer_1 = 512 
hidden_layer_2 = 256   

initial_lr = 5e-4 # base lr


online_enc_proj  = encoder_projector(l2_reg_penalty = l2_reg_penalty, input_shape = (IMG_H,IMG_W,3), hid_1 = hidden_layer_1, hid_2 = hidden_layer_2)
target_enc_proj  = encoder_projector(l2_reg_penalty = l2_reg_penalty, input_shape = (IMG_H,IMG_W,3), hid_1 = hidden_layer_1, hid_2 = hidden_layer_2)
online_predictor = predictor(input_dim = hidden_layer_2, l2_reg_penalty = l2_reg_penalty, hid_1 = hidden_layer_1, hid_2 = hidden_layer_2)

epoch_wise_loss = []
valid_epoch_loss = []
lr_epoch = []


print("Decay Steps : ",decay_steps)
lr_decayed_fn = tf.keras.experimental.CosineDecay(initial_learning_rate = initial_lr, 
                                                decay_steps = decay_steps)

cosine_with_warmUp = WarmUp(initial_learning_rate = initial_lr,
                          decay_schedule_fn = lr_decayed_fn,
                          warmup_steps = warmup_steps)

optimizer = tf.keras.optimizers.Adam(cosine_with_warmUp)

online_enc_proj.summary()

MobileNetV1_L2_10_6_warmUp_BYOL_batchNorm_colorJ_0.1_op_1024_random_max_pool_minObjCov_0.7
Decay Steps :  390000
Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_9 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 mobilenet_1.00_32 (Function  (None, 1, 1, 1024)       3228864   
 al)                                                             
                                                                 
 global_max_pooling2d_2 (Glo  (None, 1024)             0         
 balMaxPooling2D)                                                
                                                                 
 model_5 (Functional)        (None, 256)               658176    
                                                                 
Total params: 3,887,040
Trainable params: 3,864,128
Non-trainable params: 22,9

In [16]:
cnt = 0
layer_names = ["conv_pw_12/kernel:0", "conv_pw_13/kernel:0"]
index_to_layer = {}
for i in online_enc_proj.layers:
    for j in i.trainable_weights:
        index_to_layer[cnt] = j.name
        cnt+=1
layer_to_index = {j:i for i,j in index_to_layer.items()}

train_byol(online_enc_proj,
           online_predictor,
           target_enc_proj,
           final_train_ds,
           optimizer,
           criterion,
           lr_epoch,
           epochs = 1)

Number of steps :  390


100%|████████████████████████████████████████████████████████████████████████████████| 390/390 [02:22<00:00,  2.73it/s]



 epoch: 1 | train loss: 1.23066950 | lr : 0.00010000001202570274 | 2.3788 mins


[1.2306695]